In [33]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
from pytenet.operation_thc import apply_thc_mpo_and_compress, add_mps_and_compress
from pytenet.operation import vdot, add_mps
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
import scipy.sparse.linalg as spla
from pytenet.krylov_time_evo import ED_time_evo, Krylov_evo_using_vecs_single_step, Krylov_time_evo_using_vecs, Krylov_evo_using_built_space, create_Krylov_space, gram_schmidt

In [34]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


In [35]:
#ref Hamiltonian as matrix
H_ref = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_thc = scipy.io.mmread('../data_water/H_water_thc.mtx').tocsr()

#initial state
filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground_ionization.pkl"
with open(filename, 'rb') as file:
    initial_state = pickle.load(file)

#thc mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)
r_THC = int((len(H_mu_nu_list_spin_layer)-1)**(1/2) / 2)
    

In [36]:
N_krylov = 4

psi_krylov_ref = Krylov_time_evo_using_vecs(H_ref, N_krylov, initial_state.as_vector(), 1, 0.03)
psi_ed = ED_time_evo(H_ref, initial_state.as_vector(), 0.03)
print('total swift:', norm(psi_ed - initial_state.as_vector()))
print(norm(psi_krylov_ref - psi_ed))

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/krylov_time_evo.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  TN[i, j] = np.vdot(Krylov_vector_list[i], H@Krylov_vector_list[j])


[[-8.19779895e-01+5.72583778e-01j]
 [ 6.25612246e-03+8.30175739e-03j]
 [ 5.19439204e-04-6.36076473e-04j]
 [-8.09283115e-05-4.25433979e-05j]]
total swift: 1.9077630331028186
5.4866900147348975e-06


In [37]:
'''
for i in range(n):
first update foldername
generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)
then calculate time-evolved state (with re-ortho)
'''
#不需要创造很多文件夹储存文件，对每个文件好好命名然后都存在同一个文件夹即可

'\nfor i in range(n):\nfirst update foldername\ngenerate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)\nthen calculate time-evolved state (with re-ortho)\n'

In [38]:
##only use max_bond, don't set truncation tol!
space_test = create_Krylov_space(4, H_mu_nu_list_spin_layer, copy.deepcopy(initial_state), 0, 30, r_THC)

[1, 4, 16, 31, 31, 16, 4, 1]
[1, 4, 16, 30, 30, 16, 4, 1]
[1, 4, 16, 30, 30, 16, 4, 1]
[1, 4, 16, 30, 30, 16, 4, 1]


In [39]:
# space_test_vec = []
# for i in range (len(space_test)):
#     temp = space_test[i].as_vector()
#     temp /= norm(temp)
#     space_test_vec.append(temp)

# space_test_vec = gram_schmidt(space_test_vec)

space_test_vec = [mps_krylov.as_vector() for mps_krylov in space_test]
space_test_vec = gram_schmidt(space_test_vec)

In [40]:
#only use max_bond, don't set truncation tol!
vec_test = Krylov_evo_using_built_space(H_ref, space_test_vec, 0.03)
print(norm(vec_test - psi_krylov_ref))
print(norm(vec_test - psi_ed))

0.0004761841209985205
0.00047618669594564364


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/krylov_time_evo.py:181: ComplexWarning: Casting complex values to real discards the imaginary part
  TN[i, j] = np.vdot(Krylov_vector_list[i], H @Krylov_vector_list[j])


In [ ]:
# dt = 0.03

# D = 40:
# 7.159356839258043e-13 (trunc error)
# 5.486690011281797e-06 (total error)

# D = 35
# 3.537471270532817e-06 (trunc error)
# 6.522437168647089e-06 (total error)

# D= 30
# 0.0004761841209985205 (trunc error)
# 0.00047618669594564364 (total error)